In [1]:
import importlib
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
from datetime import datetime
from datetime import date
from datetime import timedelta

In [2]:
import PrjtCF_module as cf

# Setting

In [3]:
pd.options.display.max_columns = 40
pd.options.display.max_rows = 200
print(pd.get_option("display.max_columns"), pd.get_option("display.max_rows"))

40 200


# Outline
* 주거시설 등 개발과 동시에 분양을 하는 사업의 재무모델
* PF대출을 통해 토지비 및 공사비 등 개발사업 자금 조달
* 금융비용을 PF대출금으로 충당
* 분양대금의 일부는 사업비로, 일부는 대출금 상환재원으로 사용
* PF대출금은 필요에 따라 한도대로 인출

# Input Index Data

In [4]:
# Setting period index
# prjt index : total project periods, and project base index
# cstrn index : construction index. Period from the construction to the completion.
# loan index : loan period index. Period from the loan execution date to the maturity.
# sales index : sales period index. Period from the sales start date to the end date.
idx = cf.PrjtIndex(idxname=['prjt', 'cstrn', 'loan', 'sales'],
                   start=['2021-11', '2021-12', '2021-11', '2021-12'],
                   periods=[26+1, 20+1, 24+1, 23+1],
                   freq='M')

# Setting a construction progress rate
idx.prcs = Series(np.ones(len(idx.cstrn)) / len(idx.cstrn),
                  index=idx.cstrn.index)

# Input Financing Condition Data

In [5]:
loan = cf.Intlz_loan(idx, idx.loan,
                  title = ['tra', 'trb'],
                  amt_ntnl=[65_000, 20_000],
                  rate_fee = [0.01, 0.03],
                  rate_IR = [0.05, 0.07])

# Input Sales Data

In [6]:
# Cash inflow assumption on sales contract
class Asmptn_sls_cls:
    pass
astn_sls = Asmptn_sls_cls()

astn_sls.cshA = [[0, 0.1],
                 [4, 0.1],
                 [8, 0.1],
                 [12, 0.1],
                 [16, 0.1],
                 [21, 0.5]]

astn_sls.cshB = [[0, 0.1],
                 [5, 0.1],
                 [10, 0.1],
                 [15, 0.1],
                 [21, 0.6]]

astn_sls.ctrtA = [[0, 0.1],
                  [2, 0.1],
                  [4, 0.1],
                  [6, 0.1],
                  [8, 0.1],
                  [10, 0.1],
                  [12, 0.1],
                  [14, 0.1],
                  [16, 0.1],
                  [18, 0.1]]

In [7]:
sales = cf.Intlz_sales_sellinlots(idx, idx.sales, 
           title=['salesA', 'salesB'], 
           amt=[110_000, 30_000],
           csh_scdd=[astn_sls.cshA, astn_sls.cshB],
           ctrt_scdd=[astn_sls.ctrtA, astn_sls.ctrtA])

# Input Cost Data

In [8]:
cost = cf.Intlz_cost(idx)
cost.inptcst('lnd',
             idx.cstrn[0],
             30_000)
cost.inptcst('cstrn',
             idx.cstrn.index,
             80_000 * idx.prcs)

cost.inptcst('slsfee',
             sales.salesA.ctrt_plan.index,
             sales.salesA.ctrt_plan * 0.03,
             feerateA=0.03)
cost.inptcst('slsfee',
             sales.salesB.ctrt_plan.index,
             sales.salesB.ctrt_plan * 0.05,
             feerateB=0.05)

# Execution Cash Flow

In [15]:
sales.salesA.df

,add_scdd,sub_scdd,bal_strt,amt_add,amt_sub,bal_end
2021-11-30,0.0,0.0,0.0,0.0,0.0,0.0
2021-12-31,0.0,0.0,0.0,0.0,0.0,0.0
2022-01-31,0.0,0.0,0.0,0.0,0.0,0.0
2022-02-28,0.0,0.0,0.0,0.0,0.0,0.0
2022-03-31,0.0,0.0,0.0,0.0,0.0,0.0
2022-04-30,0.0,0.0,0.0,0.0,0.0,0.0
2022-05-31,0.0,0.0,0.0,0.0,0.0,0.0
2022-06-30,0.0,0.0,0.0,0.0,0.0,0.0
2022-07-31,0.0,0.0,0.0,0.0,0.0,0.0
2022-08-31,0.0,0.0,0.0,0.0,0.0,0.0
